In [1]:
from __future__ import print_function, division

In [2]:
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys

import numpy as np

Using TensorFlow backend.


In [3]:
class GAN():
    def __init__(self):
        self.img_rows = 28 
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy', 
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build and compile the generator
        self.generator = self.build_generator()
        self.generator.compile(loss='binary_crossentropy', optimizer=optimizer)

        # The generator takes noise as input and generated imgs
        z = Input(shape=(100,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The valid takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator) takes
        # noise as input => generates images => determines validity 
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):

        noise_shape = (100,)
        
        model = Sequential()

        model.add(Dense(256, input_shape=noise_shape))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=noise_shape)
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        img_shape = (self.img_rows, self.img_cols, self.channels)
        
        model = Sequential()

        model.add(Flatten(input_shape=img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, save_interval=50):

        # Load the dataset
        (X_train, _), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)

        half_batch = int(batch_size / 2)

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half batch of images
            idx = np.random.randint(0, X_train.shape[0], half_batch)
            imgs = X_train[idx]

            noise = np.random.normal(0, 1, (half_batch, 100))

            # Generate a half batch of new images
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)


            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.normal(0, 1, (batch_size, 100))

            # The generator wants the discriminator to label the generated samples
            # as valid (ones)
            valid_y = np.array([1] * batch_size)

            # Train the generator
            g_loss = self.combined.train_on_batch(noise, valid_y)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                self.save_imgs(epoch)

    def save_imgs(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, 100))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("gan/images/mnist_%d.png" % epoch)
        plt.close()

In [ ]:
if __name__ == '__main__':
    gan = GAN()
    gan.train(epochs=30000, batch_size=32, save_interval=200)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
_________________________________________________________________
____

95 [D loss: 0.010066, acc.: 100.00%] [G loss: 4.732073]
96 [D loss: 0.008794, acc.: 100.00%] [G loss: 4.496147]
97 [D loss: 0.012226, acc.: 100.00%] [G loss: 4.603278]
98 [D loss: 0.009165, acc.: 100.00%] [G loss: 4.645370]
99 [D loss: 0.027020, acc.: 100.00%] [G loss: 4.974081]
100 [D loss: 0.016415, acc.: 100.00%] [G loss: 4.606877]
101 [D loss: 0.005322, acc.: 100.00%] [G loss: 4.659884]
102 [D loss: 0.028180, acc.: 100.00%] [G loss: 4.901446]
103 [D loss: 0.015388, acc.: 100.00%] [G loss: 5.020448]
104 [D loss: 0.023583, acc.: 100.00%] [G loss: 4.878134]
105 [D loss: 0.015537, acc.: 100.00%] [G loss: 5.016105]
106 [D loss: 0.007816, acc.: 100.00%] [G loss: 4.871649]
107 [D loss: 0.019241, acc.: 100.00%] [G loss: 4.870502]
108 [D loss: 0.043151, acc.: 96.88%] [G loss: 3.948318]
109 [D loss: 0.032508, acc.: 100.00%] [G loss: 4.364313]
110 [D loss: 0.011687, acc.: 100.00%] [G loss: 4.665498]
111 [D loss: 0.014660, acc.: 100.00%] [G loss: 4.907797]
112 [D loss: 0.032230, acc.: 100.00%]

241 [D loss: 0.492710, acc.: 68.75%] [G loss: 2.081997]
242 [D loss: 0.145557, acc.: 93.75%] [G loss: 2.446065]
243 [D loss: 0.298169, acc.: 90.62%] [G loss: 3.236082]
244 [D loss: 0.342741, acc.: 81.25%] [G loss: 2.356487]
245 [D loss: 0.226810, acc.: 87.50%] [G loss: 2.371644]
246 [D loss: 0.387323, acc.: 78.12%] [G loss: 3.689922]
247 [D loss: 1.273152, acc.: 37.50%] [G loss: 1.140172]
248 [D loss: 0.342898, acc.: 78.12%] [G loss: 3.013221]
249 [D loss: 0.325710, acc.: 90.62%] [G loss: 2.073504]
250 [D loss: 0.186745, acc.: 93.75%] [G loss: 2.976958]
251 [D loss: 0.386686, acc.: 87.50%] [G loss: 2.068966]
252 [D loss: 0.337816, acc.: 87.50%] [G loss: 3.104889]
253 [D loss: 0.765719, acc.: 65.62%] [G loss: 2.245585]
254 [D loss: 0.237453, acc.: 90.62%] [G loss: 2.930868]
255 [D loss: 0.491795, acc.: 71.88%] [G loss: 1.929404]
256 [D loss: 0.418645, acc.: 78.12%] [G loss: 2.742939]
257 [D loss: 1.059082, acc.: 37.50%] [G loss: 0.877266]
258 [D loss: 0.325974, acc.: 81.25%] [G loss: 2.

391 [D loss: 0.709370, acc.: 40.62%] [G loss: 0.694999]
392 [D loss: 0.678035, acc.: 46.88%] [G loss: 0.690544]
393 [D loss: 0.665237, acc.: 46.88%] [G loss: 0.694841]
394 [D loss: 0.637839, acc.: 56.25%] [G loss: 0.713867]
395 [D loss: 0.744702, acc.: 31.25%] [G loss: 0.662668]
396 [D loss: 0.683083, acc.: 46.88%] [G loss: 0.689856]
397 [D loss: 0.695734, acc.: 46.88%] [G loss: 0.684317]
398 [D loss: 0.658726, acc.: 53.12%] [G loss: 0.677610]
399 [D loss: 0.649472, acc.: 50.00%] [G loss: 0.697524]
400 [D loss: 0.657931, acc.: 46.88%] [G loss: 0.706568]
401 [D loss: 0.710896, acc.: 43.75%] [G loss: 0.685812]
402 [D loss: 0.649643, acc.: 53.12%] [G loss: 0.677892]
403 [D loss: 0.644775, acc.: 43.75%] [G loss: 0.712564]
404 [D loss: 0.676813, acc.: 43.75%] [G loss: 0.692668]
405 [D loss: 0.656954, acc.: 50.00%] [G loss: 0.696992]
406 [D loss: 0.726089, acc.: 34.38%] [G loss: 0.694965]
407 [D loss: 0.708975, acc.: 43.75%] [G loss: 0.680317]
408 [D loss: 0.686896, acc.: 43.75%] [G loss: 0.

540 [D loss: 0.677906, acc.: 50.00%] [G loss: 0.697355]
541 [D loss: 0.654985, acc.: 53.12%] [G loss: 0.684628]
542 [D loss: 0.659584, acc.: 59.38%] [G loss: 0.681251]
543 [D loss: 0.675190, acc.: 59.38%] [G loss: 0.669322]
544 [D loss: 0.659127, acc.: 56.25%] [G loss: 0.671758]
545 [D loss: 0.644861, acc.: 56.25%] [G loss: 0.681894]
546 [D loss: 0.680914, acc.: 46.88%] [G loss: 0.666532]
547 [D loss: 0.678229, acc.: 53.12%] [G loss: 0.662831]
548 [D loss: 0.665408, acc.: 53.12%] [G loss: 0.664256]
549 [D loss: 0.688268, acc.: 53.12%] [G loss: 0.655240]
550 [D loss: 0.653435, acc.: 53.12%] [G loss: 0.650672]
551 [D loss: 0.646093, acc.: 56.25%] [G loss: 0.656877]
552 [D loss: 0.693759, acc.: 46.88%] [G loss: 0.662821]
553 [D loss: 0.651468, acc.: 53.12%] [G loss: 0.658551]
554 [D loss: 0.671319, acc.: 50.00%] [G loss: 0.657091]
555 [D loss: 0.620701, acc.: 68.75%] [G loss: 0.633899]
556 [D loss: 0.659760, acc.: 59.38%] [G loss: 0.661537]
557 [D loss: 0.669057, acc.: 56.25%] [G loss: 0.

...

27931 [D loss: 0.723307, acc.: 43.75%] [G loss: 0.789305]
27932 [D loss: 0.650387, acc.: 68.75%] [G loss: 0.827460]
27933 [D loss: 0.724711, acc.: 43.75%] [G loss: 0.766797]
27934 [D loss: 0.688585, acc.: 56.25%] [G loss: 0.855259]
27935 [D loss: 0.659600, acc.: 56.25%] [G loss: 0.822003]
27936 [D loss: 0.657466, acc.: 71.88%] [G loss: 0.842808]
27937 [D loss: 0.629280, acc.: 62.50%] [G loss: 0.810383]
27938 [D loss: 0.617771, acc.: 65.62%] [G loss: 0.821271]
27939 [D loss: 0.702542, acc.: 56.25%] [G loss: 0.765473]
27940 [D loss: 0.645595, acc.: 53.12%] [G loss: 0.781618]
27941 [D loss: 0.658975, acc.: 65.62%] [G loss: 0.812709]
27942 [D loss: 0.794356, acc.: 46.88%] [G loss: 0.767412]
27943 [D loss: 0.714037, acc.: 50.00%] [G loss: 0.747690]
27944 [D loss: 0.665390, acc.: 68.75%] [G loss: 0.827033]
27945 [D loss: 0.659938, acc.: 62.50%] [G loss: 0.869236]
27946 [D loss: 0.745992, acc.: 43.75%] [G loss: 0.888146]
27947 [D loss: 0.685588, acc.: 46.88%] [G loss: 0.883987]
27948 [D loss:

28076 [D loss: 0.613617, acc.: 71.88%] [G loss: 0.937959]
28077 [D loss: 0.640636, acc.: 71.88%] [G loss: 0.916433]
28078 [D loss: 0.628159, acc.: 68.75%] [G loss: 0.877708]
28079 [D loss: 0.638498, acc.: 62.50%] [G loss: 0.850159]
28080 [D loss: 0.745814, acc.: 56.25%] [G loss: 0.892405]
28081 [D loss: 0.652176, acc.: 62.50%] [G loss: 0.779782]
28082 [D loss: 0.696052, acc.: 59.38%] [G loss: 0.766048]
28083 [D loss: 0.570567, acc.: 78.12%] [G loss: 0.844299]
28084 [D loss: 0.650918, acc.: 56.25%] [G loss: 0.790672]
28085 [D loss: 0.723309, acc.: 56.25%] [G loss: 0.806999]
28086 [D loss: 0.659134, acc.: 56.25%] [G loss: 0.799461]
28087 [D loss: 0.685126, acc.: 46.88%] [G loss: 0.895276]
28088 [D loss: 0.747903, acc.: 50.00%] [G loss: 0.776080]
28089 [D loss: 0.731773, acc.: 37.50%] [G loss: 0.795136]
28090 [D loss: 0.701543, acc.: 50.00%] [G loss: 0.783226]
28091 [D loss: 0.694591, acc.: 46.88%] [G loss: 0.880642]
28092 [D loss: 0.740212, acc.: 43.75%] [G loss: 0.854254]
28093 [D loss:

28221 [D loss: 0.686570, acc.: 65.62%] [G loss: 0.833671]
28222 [D loss: 0.630484, acc.: 68.75%] [G loss: 0.876058]
28223 [D loss: 0.623978, acc.: 71.88%] [G loss: 0.820531]
28224 [D loss: 0.642160, acc.: 68.75%] [G loss: 0.842627]
28225 [D loss: 0.799526, acc.: 34.38%] [G loss: 0.804200]
28226 [D loss: 0.596130, acc.: 75.00%] [G loss: 0.819626]
28227 [D loss: 0.665745, acc.: 62.50%] [G loss: 0.865242]
28228 [D loss: 0.599552, acc.: 71.88%] [G loss: 0.884574]
28229 [D loss: 0.701929, acc.: 46.88%] [G loss: 0.884251]
28230 [D loss: 0.687766, acc.: 53.12%] [G loss: 0.890365]
28231 [D loss: 0.655679, acc.: 62.50%] [G loss: 0.859249]
28232 [D loss: 0.779186, acc.: 50.00%] [G loss: 0.855834]
28233 [D loss: 0.637255, acc.: 65.62%] [G loss: 0.850562]
28234 [D loss: 0.716673, acc.: 46.88%] [G loss: 0.864227]
28235 [D loss: 0.710393, acc.: 53.12%] [G loss: 0.812924]
28236 [D loss: 0.701579, acc.: 56.25%] [G loss: 0.808459]
28237 [D loss: 0.695267, acc.: 53.12%] [G loss: 0.794681]
28238 [D loss:

28363 [D loss: 0.663791, acc.: 50.00%] [G loss: 0.903134]
28364 [D loss: 0.724463, acc.: 46.88%] [G loss: 0.860610]
28365 [D loss: 0.634943, acc.: 62.50%] [G loss: 0.936282]
28366 [D loss: 0.736827, acc.: 43.75%] [G loss: 0.884625]
28367 [D loss: 0.658895, acc.: 71.88%] [G loss: 0.804107]
28368 [D loss: 0.611642, acc.: 62.50%] [G loss: 0.792758]
28369 [D loss: 0.662020, acc.: 62.50%] [G loss: 0.817275]
28370 [D loss: 0.660653, acc.: 62.50%] [G loss: 0.772285]
28371 [D loss: 0.608473, acc.: 59.38%] [G loss: 0.842323]
28372 [D loss: 0.675123, acc.: 56.25%] [G loss: 0.887778]
28373 [D loss: 0.706510, acc.: 62.50%] [G loss: 0.837301]
28374 [D loss: 0.615074, acc.: 59.38%] [G loss: 0.857274]
28375 [D loss: 0.662007, acc.: 56.25%] [G loss: 0.844339]
28376 [D loss: 0.790941, acc.: 43.75%] [G loss: 0.852735]
28377 [D loss: 0.592683, acc.: 65.62%] [G loss: 0.870853]
28378 [D loss: 0.650692, acc.: 65.62%] [G loss: 0.889714]
28379 [D loss: 0.697587, acc.: 53.12%] [G loss: 0.879964]
28380 [D loss:

28506 [D loss: 0.673349, acc.: 53.12%] [G loss: 0.795818]
28507 [D loss: 0.728013, acc.: 50.00%] [G loss: 0.802234]
28508 [D loss: 0.758528, acc.: 37.50%] [G loss: 0.812679]
28509 [D loss: 0.714613, acc.: 53.12%] [G loss: 0.840873]
28510 [D loss: 0.653132, acc.: 62.50%] [G loss: 0.782129]
28511 [D loss: 0.669567, acc.: 53.12%] [G loss: 0.832613]
28512 [D loss: 0.689986, acc.: 46.88%] [G loss: 0.945869]
28513 [D loss: 0.701718, acc.: 59.38%] [G loss: 0.881280]
28514 [D loss: 0.672933, acc.: 59.38%] [G loss: 0.885345]
28515 [D loss: 0.649738, acc.: 68.75%] [G loss: 0.799870]
28516 [D loss: 0.671474, acc.: 53.12%] [G loss: 0.804753]
28517 [D loss: 0.701964, acc.: 59.38%] [G loss: 0.853360]
28518 [D loss: 0.607097, acc.: 75.00%] [G loss: 0.802598]
28519 [D loss: 0.652075, acc.: 56.25%] [G loss: 0.900859]
28520 [D loss: 0.697761, acc.: 50.00%] [G loss: 0.835962]
28521 [D loss: 0.680071, acc.: 62.50%] [G loss: 0.883586]
28522 [D loss: 0.632821, acc.: 59.38%] [G loss: 0.810010]
28523 [D loss: